In [6]:
from math import log
def calShannoEnt (dataSet):
    numEntries = len(dataSet)
    labelCounts= {}
    for featVec in dataSet:
        currentLable = featVec[-1]
        if  currentLable not in labelCounts.keys():
            labelCounts[currentLable] = 0
        labelCounts[currentLable] += 1
    shannoEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannoEnt -= prob*log(prob, 2)
    return shannoEnt

In [2]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values
    return dataSet, labels

In [8]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        #按照 featVec[axis] 来划分数据集
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            #去除 featVec[axis] 的影响，然后加入到 retDataSet 中
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [64]:
myData, labels = createDataSet()
myData
labels

['no surfacing', 'flippers']

In [10]:
splitDataSet(myData, 0, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [11]:
splitDataSet(myData, 0, 0)

[[1, 'no'], [1, 'no']]

In [52]:
def chooseFeatureToSplit(dataSet):
    #最后一个是 label 所以减1
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calShannoEnt(dataSet)
    bestInfoGain = 0.0 
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList) 
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calShannoEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature  

In [35]:
chooseFeatureToSplit(myData)

2
infoGain is 0.419973 and feature is 0
infoGain is 0.170951 and feature is 1


0

In [36]:
import operator
def majorityCnt (classList):
    classCount = {}
    for vote in classList:
        if vote not in classList.keys():
            classCount [vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems() , key = operator.itemgetter(1), reverse = True)
    return  sortedClassCount[0][0]

In [78]:
def createTree(dataSet, labels):
    classList = [example[-1] for example  in dataSet]
    # 只有 classList[0] 这一个类别
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    # 遍历到最后一个特征
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat = chooseFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat] 
    myTree = {bestFeatLabel: {}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat,value), subLabels)
    return myTree

In [79]:
myData, labels = createDataSet()
tree = createTree(myData, labels)
print (tree)

2
1
{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
